# C&A ArcFace Competition

Seunghun Paik

# Import Tools
- model: backbone models
- header: headers
- data_process: dataset & dataloader

In [ ]:
from model import *
from header import *
from data_process import *
from torch.utils.data import DataLoader

In [ ]:
# Device choice
if torch.cuda.is_available():
    # Use gpu(0)
    torch.cuda.set_device(0)
    print("Gpu #%d activated..."%torch.cuda.current_device())
    device = torch.device('cuda')
else: device = torch.device('cpu')

# Hyperparameter Management
Overall Hyperparameters, just like as `config`

In [ ]:
# Image folder path
img_dir = './faces_dataset/'
idx_path = 'train.idx'
rec_path = 'train.rec'

# Dataset & Training Loop
img_shape = (3,112,112)
batch_size = 64
epoch = 34

# margins for arcface/cosface
emb_size = 512
scale = 64
margin = 0.5

# Optimizer
lr_backbone = 0.05
lr_header = 0.05
weight_decay = 2e-4
beta1, beta2 = 0.9, 0.999

# Scheduler
gam_backbone = 0.93
gam_header = 0.85

# Get Dataset & DataLoader
- Casia_WebFace is used
- Dataset & DataLoader Implemented
- Load data on cpu (RAM) / allocate to gpu at train phase

In [ ]:
train_dataset = get_dataset(img_dir, idx_path, rec_path)
dataloader = DataLoader(train_dataset, batch_size, 
                        shuffle = True, drop_last = True)

num_classes = train_dataset.num_classes
img_shape = train_dataset.shape
num_imgs = train_dataset.__len__()
#Test; use 10% of the total dataset.
#num_imgs =  50000
print("# of classes : ", num_classes)
print("Image shape : ", img_shape)
print("# of images : ", num_imgs)

In [ ]:
# Visualization of the input image
import matplotlib.pyplot as plt
img, label = train_dataset.__getitem__(239469)
img_numpy = ((img+1.)/2.).numpy().transpose(1,2,0)
plt.imshow(img_numpy)
plt.show()

# Import Model and Allocate to Device
- Resnet => 18,34,50,100 supported
- header => plain, cosface, arcface, supported
- adacos, partialfc, magface => Not Implemented Yet.

In [ ]:
# Configuration for Backbone and Header
num_backbone_layer = 50
header_type = "arcface"

In [ ]:
# Get Backbone and Header
backbone = get_resnet(num_backbone_layer, emb_size)
header = get_header(header_type, emb_size=emb_size,
                    num_classes=num_classes,
                    scale=scale, margin=margin)

In [ ]:
# Feed parameters from RAM to GPU
backbone = backbone.to(device)
header = header.to(device)

# Optimizer and LR scheduler
SGD w/ Momentum

- learning rate : 0.05
- weight decay : 2e-4
- momentum : 0.9 
- batch_size = 64
- Image: 112x112 => 512D vector

In [ ]:
optim_type = 'sgd'

if optim_type == "adam":
    optim_backbone = torch.optim.Adam(backbone.parameters(), lr = lr_backbone,
                                betas = [beta1,beta2], weight_decay = weight_decay)
    optim_header = torch.optim.Adam(header.parameters(), lr=lr_header,
                              betas = [beta1, beta2], weight_decay = weight_decay)
    
if optim_type == "sgd":
    optim_backbone = torch.optim.SGD(backbone.parameters(), lr = lr_backbone,
                                    momentum = 0.9, weight_decay = weight_decay)
    optim_header = torch.optim.SGD(header.parameters(), lr= lr_header,
                                  momentum = 0.9, weight_decay = weight_decay)

In [ ]:
# Similar LR schedule policy as original paper / Continuously changing LR
# Header =>  LR shrinks faster

sched_type = 'exp'

if sched_type == 'exp':
    sched_backbone = torch.optim.lr_scheduler.ExponentialLR(optim_backbone,
                                                       gam_backbone)
    sched_header = torch.optim.lr_scheduler.ExponentialLR(optim_header,
                                                     gam_header)
    
if sched_type == 'step':
    sched_backbone = torch.optim.lr_scheduler.MultiStepLR(optim_backbone,
                                                         milestones = [10,17,23,27,31], gamma=0.35)
    sched_header = torch.optim.lr_scheduler.MultiStepLR(optim_header,
                                                       milestones = [10,15,20,25,30])

# Train & Evaluation Code
To measure the time consumed, module `time` is used

In [ ]:
import time
from verification import *

In [ ]:
dataset_lfw = load_bin(img_dir+"/lfw.bin", (112,112))

In [ ]:
#Save Current Parameters

def save_model_parameters(model, name):
    dir_header = "./params/"
    torch.save(model.state_dict(), dir_header+name+'_params.pth')
    
    
def save_model(model, name):
    dir_header = "./params/"
    torch.save(model, dir_header+name+'_model.pth')

In [ ]:
def train(dataloader, epoch, backbone, header, 
          optim_header, optim_backbone,
         sched_backbone, sched_header, tag):
    size = len(dataloader.dataset)
    dir_prefix = "/params/"
    dir_suffix = ".pth"
    acc_track = [0] * epoch
    loss_track = [0] * epoch
    
    state = tag+"_started."
    logging('log.txt', state)
    
    for ech in range(epoch):
        tic_ech = time.time()
        tic_batch = time.time()
        
        cnt_ans = 0
        
        print("epoch {} begins".format(ech+1))
        print("Learning rate backbone: {}".format(optim_backbone.param_groups[0]['lr']))
        print("Learning rate header: {}".format(optim_header.param_groups[0]['lr']))
        
        for idx,(x,y) in enumerate(dataloader):
            x,y = x.to(device), y.to(device)
            
            # Forward_pass
            feature = backbone(x)
            loss = header(feature,y)

            # Backward_pass
            optim_header.zero_grad()
            optim_backbone.zero_grad()
            loss.backward()
            optim_header.step()
            optim_backbone.step()
            
            # Logging for 200 steps
            if idx % 200 == 0 and idx !=0:
                toc_batch = time.time()
                
                loss, current = loss.item(), idx * batch_size
                print(f"loss: {loss:>7f}  [{current:>5d}/{num_imgs:>5d}]")
                print("time consumed : ", toc_batch - tic_batch)

                
                tic_batch = time.time()
                
                state = "Epoch#{}_Iter#{}_loss: {:.3f}".format(ech+1, current,loss)
                logging("log.txt",state)
            
            if idx * batch_size > num_imgs:
                break
                
        
        toc_ech = time.time()
        
        # Logging for each epoch
        print()
        print("epoch %d done."%(ech+1))
        print("time consumed : ", toc_ech-tic_ech)
   
        sched_backbone.step()
        sched_header.step()
        
        acc1, std1, acc2, std2 ,xnorm, _ = test(dataset_lfw, 
                                   backbone, batch_size, device, nfolds=10)
        
        print("[Accuracy: {:.3f}+-{:3f}]".format(acc2*100,std2*100))
        print("[Xnorm: {:.3f}]".format(xnorm))
        print()
        
        state = "Epoch#{} end, Accuracy: {:.3f}, Time Consumed: {:.3f}".format(ech+1, acc2*100, toc_ech-tic_ech)
        logging("log.txt",state)
        
        if ech%5 ==0 and ech>0:
            save_model(backbone, header_type + "_"+tag+ "_backbone_%d"%(ech+1))
            save_model(header, header_type + "_"+tag+"_header_%d"%(ech+1))
        acc_track[ech] = acc2
        loss_track[ech] = loss
        
    return acc_track, loss_track
        

In [ ]:
# Logging
import datetime

def logging(name, state):
    curr_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    f = open(name, 'a')
    f.write("[{}] ".format(curr_time)+ state+"\n")
    f.close()
    
    

In [ ]:
# Tracking computed accuracy and loss
acc_track, loss_track = train(dataloader, epoch, backbone, header, optim_header, optim_backbone,
     sched_backbone, sched_header, tag="arc_res50_adam_0.05_step_do")

In [ ]:
# Visualization Tools
def viz_loss(loss_track, acc_track):
    losses = [loss.item() for loss in loss_track]
    acces = acc_track
    plt.subplot(1,2,1)
    plt.plot(losses)
    
    plt.subplot(1,2,2)
    plt.plot(acces)
    
    plt.show()

In [ ]:
viz_loss(loss_track, acc_track)

# Visualization Tool
Use MatPlotLib to Visualize the model result

In [ ]:
import matplotlib.pyplot as plt
import math

In [ ]:
# Visualiazation Tools
import random

@torch.no_grad()
def visualize(img1, img2):
    # img: 1 X 3 X 112 X 112 / torch.Tonsor
    # Feature extraction => Angular Distance Check
    
    img1 = img1.to(device)
    img2 = img2.to(device)
    backbone.eval()
    feat1 = backbone(img1.unsqueeze(0))
    feat2 = backbone(img2.unsqueeze(0))
    
    norm_feat1, norm_feat2 = F.normalize(feat1), F.normalize(feat2)
    cos_value = torch.dot(norm_feat1.squeeze(0), norm_feat2.squeeze(0))

    angular_dist = torch.acos(cos_value).item() * (180 / 3.14)
    
    img1_np = (img1.cpu().numpy().transpose(1,2,0) + 1.)/2
    img2_np = (img2.cpu().numpy().transpose(1,2,0)+1.)/2
    
    plt.subplot(1,2,1)
    plt.imshow(img1_np)
    
    plt.subplot(1,2,2)
    plt.imshow(img2_np)
    plt.show()
    print("Angular Distnace is : ", angular_dist)
    print("Cosine value is : ", cos_value.item())
    
    backbone.train()

    return 

# Get image from dataset and Visualize;
r1, r2 = random.randint(0, num_imgs), random.randint(0, num_imgs)
img1, img2 = train_dataset.__getitem__(r1), train_dataset.__getitem__(r2)
visualize(img1[0], img2[0])

# Save & Load Parameter
Model parameters are saved at "/params/". Format is ***.pth.

In [ ]:
# Load Model Parameters

def load_model_params(model, name):
    dir_header = "./params/"
    
    try:
        model.load_state_dict(torch.load(dir_header+name+'_params.pth'))
        return model
        
    except:
        print("Invalid Loading, Quitting...")
        return

def load_model(name):
    dir_header = "./params/"
    
    try:
        model = torch.load(dir_h eader + name _'_models.pth')
        return model
    
    except:
        print("Invalid Loading, Quitting...")
        return